In [1]:
!pip install pyspark 
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 36.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=51898d404612a9e6682796d522ffc6f1812fd2a3c610c79499ba12110ada1f8a
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
cur_path = "/content/drive/MyDrive/bds/"
os.chdir(cur_path)
!pwd

/content/drive/MyDrive/bds


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("netflix").getOrCreate()

In [19]:
df = spark.read.csv(cur_path + 'netflix_titles.txt',inferSchema=True,header=True,sep='\t')

In [20]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- show_id: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [22]:
df.select('director').show()

+--------------------+
|            director|
+--------------------+
|Richard Finn, Tim...|
|                null|
|                null|
|                null|
|    Fernando Lebrija|
|                null|
|         Gabe Ibáñez|
|Rodrigo Toro, Fra...|
|                null|
|   Henrik Ruben Genz|
|José Miguel Contr...|
|    Daniel Alfredson|
|                null|
|Munjal Shroff, Ti...|
|Munjal Shroff, Ti...|
|        Tilak Shetty|
|        Tilak Shetty|
|                null|
|Munjal Shroff, Ti...|
|          Gaspar Noé|
+--------------------+
only showing top 20 rows



In [23]:
import itertools
from pyspark import SparkContext

In [24]:
df = df.na.drop(subset=['director'])

In [25]:
director = df.select("director").rdd.flatMap(lambda x:x)

In [26]:
director_pair = director.map(lambda x:x.split(", ")).map(lambda x:list(itertools.combinations(x,2))).collect()

In [27]:
from pyspark.sql.types import *

In [28]:
spark = SparkSession.builder.appName('df2').getOrCreate()

In [30]:
field = [
    StructField('dir1', StringType(), True),
    StructField("dir2", StringType(), True),
]
schema = StructType(field)
df2 = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema)
df2.printSchema()

root
 |-- dir1: string (nullable = true)
 |-- dir2: string (nullable = true)



In [31]:
df2.show()

+----+----+
|dir1|dir2|
+----+----+
+----+----+



In [32]:
for i in director_pair:
  if i:
    for element in i:
      #print(type(element))
      newRow = spark.createDataFrame([sorted(element)])
      df2 = df2.union(newRow)

KeyboardInterrupt: ignored

In [33]:
file = open("director.txt","a+")
for i in director_pair:
  if i:
    for element in i:
      element_list = sorted(element)
      pair = element_list[0]+','+ element_list[1]+'\n'
      file.write(pair)
file.close()